# Imports + settings

In [1]:
import os
os.chdir("../..")  # Move one level up  
print(os.getcwd())

/Users/malthepabst/Documents/GitHub/Thesis_Neurodiversity


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scripts.eda_func import plot_small_multiple_rois, test_dist, group_roi_heat_map

In [ ]:
pd.options.mode.chained_assignment = None

In [37]:
num_of_rois = 7
roi_names = [f"ROI_{i+1}" for i in range(num_of_rois)]
meta_data = pd.read_csv(f'data.nosync/phenotypic/subjects_with_meta_{num_of_rois}.csv',
                         index_col= 'Unnamed: 0')
meta_data['Sub ID'] = meta_data['Sub ID'].apply(lambda x: str(x).zfill(7))
meta_data.shape

(660, 8)

# Participant stats

### Take-aways

- The oldest participant is a 39 years old male participant, while the youngest is a 5 years old female participant.
- The highest IQ is at 148, while the lowest is at 67.
- There is some outliers in terms of age. Especeially at 30+.
- The TD-Other has the highest median (IQ), however there is only 6 participants in this group. With one female participant, and five male participants. 
- Most of the participants are at age 15 or below.
- There does not seem to be a clear clustering for the groups (diagnosis and sex), when looking at the age and IQ plots. 


In [38]:
participants = meta_data.drop_duplicates(subset=['Sub ID', 'Dataset'])
participants['Co-Diagnosis'] = participants['Co-Diagnosis'].replace({np.nan: ''})
participants['Co-Diagnosis'] = participants['Co-Diagnosis'].replace({np.nan: ''})
participants['Co-Diagnosis'] = participants['Co-Diagnosis'].apply(lambda x: '-'+x if x != '' else x)
participants['Diagnosis'] = participants['Diagnosis'] + participants['Co-Diagnosis']
participants.groupby(['Diagnosis', 'Sex']).agg({'Sex':['count'],
                                                                'IQ':['min', 'max', 'mean', 'std'], 
                                                                'Age':['min', 'max', 'mean', 'std']})


/var/folders/dn/nd5kb8wx3xs44tlt7kst4n7h0000gn/T/ipykernel_43840/3920090657.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  participants['Co-Diagnosis'] = participants['Co-Diagnosis'].replace({np.nan: ''})
/var/folders/dn/nd5kb8wx3xs44tlt7kst4n7h0000gn/T/ipykernel_43840/3920090657.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  participants['Co-Diagnosis'] = participants['Co-Diagnosis'].replace({np.nan: ''})
/var/folders/dn/nd5kb8wx3xs44tlt7kst4n7h0000gn/T/ipykernel_43840/3920090657.py:4: SettingWit

Sex     IQ                                   Age         \
                  count    min    max        mean        std     min    max   
Diagnosis  Sex                                                                
ADHD       Female    27   78.0  128.0  106.920000  12.724386   7.350  17.15   
           Male      76   73.0  134.0  105.945946  14.895887   7.260  17.61   
ADHD-Other Female     8   75.0  127.0  107.375000  15.574131   8.610  13.24   
           Male      33   82.0  132.0  103.562500  13.452275   7.240  17.37   
ASD        Female     7   78.0  134.0  101.428571  22.618997   5.220  34.76   
           Male      50   67.0  137.0  101.740000  17.090169   5.320  39.10   
ASD-ADHD   Female     3   74.0  132.0  106.666667  29.687259  10.000  38.76   
           Male      34   75.0  148.0  110.909091  17.782301   5.928  26.62   
ASD-Other  Female     3   91.0  118.0  102.333333  14.011900   6.750  24.41   
           Male      20   80.0  142.0  110.400000  16.368776   5.920  29.18   
TD         Female    75   80.0  140.0  109.690141  14.740219   5.907  29.13   
           Male     146   80.0  144.0  113.524476  13.479683   5.887  30.78   
TD-Other   Female     1  101.0  101.0  101.000000        NaN   8.100   8.10   
           Male       5   81.0  129.0  117.400000  20.427922   7.190  16.93   

                                         
                        mean        std  
Diagnosis  Sex                           
ADHD       Female  10.351481   2.553156  
           Male    11.201842   2.744991  
ADHD-Other Female  10.936250   1.807413  
           Male    11.537576   2.739780  
ASD        Female  14.442857   9.680924  
           Male    12.828120   6.696238  
ASD-ADHD   Female  20.174667  16.119519  
           Male    10.593176   5.348535  
ASD-Other  Female  17.990000   9.766944  
           Male    13.570500   5.769673  
TD         Female  12.996893   4.821956  
           Male    13.184863   5.333637  
TD-Other   Female   8.100000        NaN  
           Male    12.868000   3.963158

In [39]:
print('Participants with no sex:',participants[participants.Sex.isna()].shape[0])
print('Participants with no age:',participants[participants.Age.isna()].shape[0])
print('Participants with no IQ:',participants[participants.IQ.isna()].shape[0])

Participants with no sex: 1
Participants with no age: 0
Participants with no IQ: 13


In [40]:
#save the ones missing in order to remove from them train, test and validation split
to_remove = pd.concat([participants[participants.Sex.isna()],
                      participants[participants.Age.isna()],
                      participants[participants.IQ.isna()]])
to_remove = to_remove[['Sub ID', 'Dataset']]
to_remove['Exclude'] = True
to_remove.to_csv(f'data.nosync/phenotypic/missing_metadata.csv')

In [41]:
participants = participants[~participants.index.isin(to_remove.index)]

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(participants[['Dataset', 'Age', 'Sex', 'IQ', 'Diagnosis']], 
                                    test_size=0.33, 
                                    random_state=42,
                                    stratify=participants[['Dataset', 'Sex', 'Diagnosis']])
X_train

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [ ]:
# Create a jointplot (initially with scatter and default histograms)
g = sns.jointplot(data=participants, x="Age", y="IQ", kind="scatter", hue= 'Diagnosis')

g.fig.suptitle("IQ vs Age by Diagnosis", fontsize=16)

for ax in g.ax_joint.collections:
    ax.set_sizes([20])

g.ax_joint.legend_.remove()

# Create a new legend outside the plot
handles, labels = g.ax_joint.get_legend_handles_labels()
g.fig.legend(
    handles=handles,
    labels=labels,
    loc='center left',
    bbox_to_anchor=(1.05, 0.5),
    title='Diagnosis'
)

# Replace marginal histograms with boxplots
g.ax_marg_x.clear()
g.ax_marg_y.clear()

g.ax_marg_x.tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)
g.ax_marg_y.tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)

# Add boxplots on margins
sns.boxplot(data=participants, x="Age", ax=g.ax_marg_x, hue= 'Diagnosis', legend = None)
sns.boxplot(data=participants, y="IQ", ax=g.ax_marg_y, hue= 'Diagnosis', legend = None)

#Remove spines
for ax in [g.ax_marg_x, g.ax_marg_y]:
    for spine in ["top", "right", "bottom", "left"]:
        ax.spines[spine].set_visible(False)

g.fig.tight_layout()
g.fig.subplots_adjust(top=0.90, right=1) 


In [ ]:
# Create a jointplot (initially with scatter and default histograms)
g = sns.jointplot(data=participants, x="Age", y="IQ", kind="scatter", hue= 'Sex')

g.fig.suptitle("IQ vs Age by Sex", fontsize=16)

for ax in g.ax_joint.collections:
    ax.set_sizes([20])

g.ax_joint.legend_.remove()

# Create a new legend outside the plot
handles, labels = g.ax_joint.get_legend_handles_labels()
g.fig.legend(
    handles=handles,
    labels=labels,
    loc='center left',
    bbox_to_anchor=(1.05, 0.5),
    title='Sex'
)

# Replace marginal histograms with boxplots
g.ax_marg_x.clear()
g.ax_marg_y.clear()

g.ax_marg_x.tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)
g.ax_marg_y.tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)

# Add boxplots on margins
sns.boxplot(data=participants, x="Age", ax=g.ax_marg_x, hue= 'Sex', legend = None)
sns.boxplot(data=participants, y="IQ", ax=g.ax_marg_y, hue= 'Sex', legend = None)

#Remove spines
for ax in [g.ax_marg_x, g.ax_marg_y]:
    for spine in ["top", "right", "bottom", "left"]:
        ax.spines[spine].set_visible(False)

g.fig.tight_layout()
g.fig.subplots_adjust(top=0.90, right=1) 

# Plot distribution of voxels

### Take-aways:
When we look at the std/mean ratio, we can see that most of the regions does not vary that much, 1-4%. However the ration for ROI 9 and 10, has a ratio of approxmiately 28\% and 33\%. When investigating ROI 5 along with the age, it is clear that there is some sort of correlation between the size of this ROI and the age.

Looking at the total number of voxels, plottet against age, we see that there is a correlation between age and the number of voxels. Mainly that the younger the subject, the more voxels there.

In [ ]:
num_of_voxels = pd.read_csv(f'data.nosync/stats/num_of_voxels_pr_timestep_{num_of_rois}.csv',
                            index_col= 'Unnamed: 0')
num_of_voxels['Sub ID'] = num_of_voxels['Sub ID'].apply(lambda x: str(x).zfill(7))
num_of_voxels['Total'] = num_of_voxels[roi_names].sum(axis = 1)
num_of_voxels = pd.merge(num_of_voxels, meta_data, how="left", on=['Sub ID', 'Dataset'])
num_of_voxels['Full Diagnosis'] = num_of_voxels['Diagnosis'] + '+' + num_of_voxels['Co-Diagnosis'].apply(lambda x: str(x))
num_of_voxels['Full Diagnosis'] = num_of_voxels['Full Diagnosis'].apply(lambda x: x.replace('+nan', ''))
num_of_voxels = num_of_voxels.sort_values('Full Diagnosis')

In [ ]:
stats_voxels = num_of_voxels[roi_names+['Total']].describe()

In [ ]:
stats_voxels = stats_voxels.T.reset_index()
stats_voxels = stats_voxels[['index', 'mean', 'std']]
#Calculate the "ratio" to make them compareable between the ROIs
# Aka is the STD proportional bigger to the mean that other ROIs
stats_voxels['ratio'] = stats_voxels['std']/stats_voxels['mean']*100
stats_voxels

In [ ]:
melted = num_of_voxels[roi_names].melt()

# setting the dimensions of the plot
p = sns.displot(data = melted,
            x='value',
            hue='variable',
            kind="kde")
p.set(title='Number of Voxels pr. ROI', xlabel= '#Voxels')
p.fig.set_figwidth(15)
p.fig.set_figheight(5)
p.tight_layout()
p.savefig(f'notebooks/pics/{num_of_rois} ROIs/Number_voxels_pr_roi_dist.png', dpi = 300)

In [ ]:
# setting the dimensions of the plot
p = sns.lmplot(data = num_of_voxels,
                    x='Age',
                    y='ROI_10',
                    truncate=True)
p.set(title='Number of Voxels in ROI 10 vs. Age', xlabel= 'Age', ylabel='#Voxels')
p.fig.set_figwidth(15)
p.fig.set_figheight(5)
p.tight_layout()
p.savefig(f'notebooks/pics/{num_of_rois} ROIs/ROI_10_voxels_age.png', dpi = 300)

In [ ]:
# setting the dimensions of the plot
p = sns.lmplot(data = num_of_voxels,
                    x='Age',
                    y='ROI_9',
                    truncate=True)
p.set(title='Number of Voxels in ROI 9 vs. Age', xlabel= 'Age', ylabel='#Voxels')
p.fig.set_figwidth(15)
p.fig.set_figheight(5)
p.tight_layout()
p.savefig(f'notebooks/pics/{num_of_rois} ROIs/ROI_9_voxels_age.png', dpi = 300)

In [ ]:
p = sns.pairplot(num_of_voxels[['Full Diagnosis','Age', 'IQ', 'Total']], hue="Full Diagnosis", plot_kws={'s': 20})

p.fig.set_figwidth(15)
p.fig.set_figheight(12)
p.tight_layout()
p.savefig(f'notebooks/pics/{num_of_rois} ROIs/Pairplot_metadata.png', dpi = 300)

# Binning and correlation

Here we calculate the means of each ROI, for different bin sizes through out the scan. E.g. we bin the scan into bins of size x, then we take the mean of the time step means for each bin. Afterwards the correlation between the ROIs means are calculated, pr. bin size. E.g calculating the correlation between the bins of size x. 

Take-aways: 
- When the bin size increases, the correlation values tends to get "polarized". This is true for all of the diagnosis. This could indicate that there might be some information in binning the timesteps, and combine the bins into multiple features.


In [ ]:
meta_data.iloc[1].file_path.replace('clean','stats').replace('bold.npz', 'b.csv')
file_list = meta_data.copy()
file_list['file_path'] = file_list['file_path'].apply(lambda x: x.replace('clean','stats').replace('bold.npz', 'b.csv'))

file_list['Co-Diagnosis'] = file_list['Co-Diagnosis'].apply(lambda x: str(x))
file_list['Full Diagnosis'] = file_list['Diagnosis'] + '+' + file_list['Co-Diagnosis']
file_list['Full Diagnosis'] = file_list['Full Diagnosis'].apply(lambda x: x.replace('+nan','').replace('TD+Other', 'TD'))

In [ ]:
bins = [1/(n+1) for n in range(17)]
#[10, 11, 12, 13, 14, 15, 16, 18, 20, 22, 25, 29, 35, 44, 59, 88, 176]
#bin_sizes = list(set([int(round(i*176)) for i in bins]))
#bin_sizes.sort()

bin_sizes = [10, 15, 20, 25, 30, 35, 45, 60, 88, 172]


#Make for loop

final_data_dict = {}

#For each diagnosis
for diag in file_list['Full Diagnosis'].unique():
    print(diag)
    temp_subs = file_list[file_list['Full Diagnosis'] == diag]
    diag_data = []
    #for each subject
    for idx, row in temp_subs.iterrows():
        test_sub = pd.read_csv(row['file_path'], delimiter = ';', index_col = 'Unnamed: 0')
        scan_length = test_sub.shape[0]
        for bin in bin_sizes:
            time = 1
            #add each bin step
            for i in range(bin, scan_length, bin+1):
                means = test_sub.iloc[i-bin:i].mean().to_dict()
                means['Time'], means['Bin'], means['Subject'], means['Dataset'], means['Full Diagnosis'] = time, bin, row['Sub ID'], row['Dataset'], diag 
                diag_data.append(means)
    
                if time == 17: 
                    print(means)
                
                time += 1
            #Add the remaning timesteps if there is at least two
            if len(test_sub.iloc[i+1:scan_length]) > 2:
                means = test_sub.iloc[i+1:scan_length].mean().to_dict()
                means['Time'], means['Bin'], means['Subject'], means['Dataset'], means['Full Diagnosis'] = time, bin, row['Sub ID'], row['Dataset'], diag 
                diag_data.append(means)
    final_data_dict[diag] = pd.DataFrame(diag_data)

#Save files
for key, value in final_data_dict.items():
    value.to_csv(f'data.nosync/stats/bins_{num_of_rois}/{key}.csv')

In [ ]:
coor_data_dict = {}
coor_mask = np.triu(np.ones((num_of_rois,num_of_rois), dtype=bool), 1)

#For each diagnosis
for key, value in final_data_dict.items():
    print(key)
    temp_dataset = final_data_dict[key]
    #Get subjects
    subjects = set(zip(temp_dataset['Subject'].tolist(), temp_dataset['Dataset'].tolist()))
    #Make list of dataframes
    subjects_data_coor = []
    #For each subject
    for sub in subjects:
        subject_data = temp_dataset[(temp_dataset['Subject'] == sub[0]) & (temp_dataset['Dataset'] == sub[1])]
        #For each bin size
        for bin in bin_sizes:
            #Calculate the correlations
            correlation_matrix = subject_data[subject_data['Bin']==bin][roi_names].corr().to_numpy()
            #Mask dataframe
            sub_coor_values = pd.DataFrame(correlation_matrix[coor_mask], columns=['Coor_value'])
            #Add bin
            sub_coor_values['bin'] = bin
            #Add to data
            subjects_data_coor.append(sub_coor_values)
    coor_data_dict[key] = pd.concat(subjects_data_coor).reset_index()

In [ ]:
for key, value in coor_data_dict.items():
    temp_data = coor_data_dict[key]
    for i in temp_data.bin.unique():
        print(f"Diagnosis: {key}, bin: {i}, variance: {temp_data[temp_data['bin'] == i]['Coor_value'].var()}")

In [ ]:
for key, value in coor_data_dict.items():
    temp_data = coor_data_dict[key]
    p = sns.displot(data = temp_data,
                x='Coor_value',
                hue='bin',
                kind="kde",
                palette= "Paired")
    p.set(title=key, xlabel= 'Correlation', ylabel='density', xlim=(-1,1))
    p.fig.set_figwidth(7)
    p.fig.set_figheight(3)
    p.tight_layout()


# Mean and variance pr. timestep

## Mean

We can see that the distributions are very similar, which makes sense since the values have been standardized in the preprocessing step. Furthermore there is only a statistical significats difference between ROI 3 and the diagnosis groups, with a p-value of 0.036. However we have not corrected for multiple hypothesis testing, e.g using the bonferroni correction, thus this is H_1 hypothesis is not accepted - aka. we do not use this hypothesis.

In [ ]:
mean_data = pd.read_csv(f'data.nosync/stats/{num_of_rois}_mean_subject.csv', delimiter= ";", index_col = 'Unnamed: 0')
mean_data['subject'] = mean_data['subject'].apply(lambda x: str(x).zfill(7))
mean_data = mean_data.sort_values(['subject', 'dataset'])
mean_data['Sex'] = meta_data.sort_values(['Sub ID', 'Dataset'])['Sex']

In [ ]:
plot_small_multiple_rois(data = mean_data, regions = roi_names , title = 'Mean', hue_col = 'diagnosis')

In [ ]:
test_dist(data = mean_data, regions = roi_names)

In [ ]:
group_roi_heat_map(data = mean_data, group = 'Sex', title = 'Mean values', regions = roi_names)

### Correlation between means difference

Calculating the correlation between the means for each of the ROIs, and substracts them from another group. E.g male vs female participants or diagnosis x vs diagnosis y. 

Take-aways: 
- Note the low sample size does affect the results.
- Between sex
    - We can see that there is differences between the correlation of the means between male participants and female participants.
    - For the TD group the differences are less clear compared to some of the other groups. Here the male participants have a higher correlation between ROI 9 and 14, 15, 16 and 17.
    - The difference between the sexes in the ADHD+Other group are more clear than the TD group. Here the male participants has higher correlation between a ROI 10 and 14, 15, 16 and 17. While the female participants has a higher correlation between ROI 4 and 7 and 9. 
- Between groups
    - We can see that there is differences between the correlation of the means between the different diagnosis (including TD).
    - The ADHD group does differer from both the TD group and ASD. Furtmore there is "more polarized" difference to be spotted between the ADHD and ASD+ADHD. This indicates that there are some differences in which ROIs are "active" (in the sense that these ROIs has mean values that correlates more than other.) at the same time. 


In [ ]:
coor_mask = np.triu(np.ones((num_of_rois,num_of_rois), dtype=bool), 1)
color_map = sns.color_palette("vlag", as_cmap=True)

groups = list(mean_data['diagnosis'].unique())

fig, axs = plt.subplots(2, 3, figsize=(15, 10), tight_layout=True)
fig.suptitle('Mean correlation difference (Male - Female)', fontsize = 20)

ax_pos = [[0,0], [0,1], [0,2], [1,0], [1,1], [1,2]]

for x in range(len(groups)):
        x_pos,y_pos = ax_pos[x]
        temp_data = mean_data[(mean_data['diagnosis'] == groups[x])]
        
        male_data = temp_data[temp_data['Sex'] == "Male"][roi_names]
        coor_male = male_data.corr()
        
        female_data = temp_data[temp_data['Sex'] == "Female"][roi_names]
        coor_female = female_data.corr()
        
        values = coor_male.to_numpy()-coor_female.to_numpy()
        max_value = values.flatten().max()
        #Create subplot
        sns.heatmap(coor_male-coor_female,
                    vmin = 1.6, #max_value,
                    vmax = -1.6, #max_value,
                    mask = coor_mask,
                    cmap = color_map, 
                    ax = axs[x_pos][y_pos])
        axs[x_pos][y_pos].set_title(f"{groups[x]} (Male: {len(male_data)}, Female: {len(female_data)})")


In [ ]:
coor_mask = np.triu(np.ones((num_of_rois,num_of_rois), dtype=bool), 1)
color_map = sns.color_palette("vlag", as_cmap=True)
mean_data['short diagnosis'] = mean_data['diagnosis'].apply(lambda x: x.replace('+Other', ''))

groups = list(mean_data['short diagnosis'].unique())

fig, axs = plt.subplots(4, 4, figsize=(25, 20), tight_layout=True)
fig.suptitle('Mean correlation difference (row - column)', fontsize = 20, y = 1)

for g1 in range(len(groups)):
        for g2 in range(len(groups)):

                group_one_data = mean_data[mean_data['short diagnosis'] == groups[g1]][roi_names].corr()
                
                group_two_data = mean_data[mean_data['short diagnosis'] == groups[g2]][roi_names].corr()
                
                values = group_one_data.to_numpy()-group_two_data.to_numpy()
                max_value = values.flatten().max()
                
                #Create subplot
                sns.heatmap(group_one_data-group_two_data,
                        vmin = max_value,
                        vmax = -max_value,
                        mask = coor_mask,
                        cmap = color_map, 
                        ax = axs[g1][g2])
                axs[g1][g2].set_title(f"{groups[g1]} - {groups[g2]}")


## Variance

There are more differene between these distributions, compared to the mean distributions. Furthermore there are many small p-values (>0.01), when looking at the ROIs and the diagnosis. These values does however have to be corrected to avoid p-hacking.

In [ ]:
var_data = pd.read_csv(f'data.nosync/stats/{num_of_rois}_variance_subject.csv', delimiter= ";", index_col = 'Unnamed: 0')
var_data['subject'] = var_data['subject'].apply(lambda x: str(x).zfill(7))
var_data = var_data.sort_values(['subject', 'dataset'])
var_data['Sex'] = meta_data.sort_values(['Sub ID', 'Dataset'])['Sex']

In [ ]:
plot_small_multiple_rois(data = var_data, regions = roi_names , title = 'Variance', hue_col = 'diagnosis')

In [ ]:
test_dist(data = var_data, regions = roi_names)

### Correlation between variance difference

Calculating the correlation between the variance for each of the ROIs, and substracts them from another group. E.g male vs female participants or diagnosis x vs diagnosis y. 

Take-aways: 
- Note the low sample size does affect the results.
- Between sex
    - Here we (like at the means) sees a difference between the sexes. 
- Between groups
    - Like means we can confirme that there is a difference between the correlation between the groups.


In [ ]:
coor_mask = np.triu(np.ones((num_of_rois,num_of_rois), dtype=bool), 1)
color_map = sns.color_palette("vlag", as_cmap=True)

groups = list(var_data['diagnosis'].unique())

fig, axs = plt.subplots(2, 3, figsize=(15, 10), tight_layout=True)
fig.suptitle('Variance correlation difference (Male - Female)', fontsize = 25)

ax_pos = [[0,0], [0,1], [0,2], [1,0], [1,1], [1,2]]

for x in range(len(groups)):
        x_pos,y_pos = ax_pos[x]
        temp_data = var_data[(var_data['diagnosis'] == groups[x])]
        
        male_data = temp_data[temp_data['Sex'] == "Male"][roi_names]
        coor_male = male_data.corr()
        
        female_data = temp_data[temp_data['Sex'] == "Female"][roi_names]
        coor_female = female_data.corr()
        
        values = coor_male.to_numpy()-coor_female.to_numpy()
        max_value = values.flatten().max()
        #Create subplot
        sns.heatmap(coor_male-coor_female,
                    vmin = max_value,
                    vmax = -max_value,
                    mask = coor_mask,
                    cmap = color_map, 
                    ax = axs[x_pos][y_pos])
        axs[x_pos][y_pos].set_title(f"{groups[x]} (Male: {len(male_data)}, Female: {len(female_data)})")


In [ ]:
coor_mask = np.triu(np.ones((num_of_rois,num_of_rois), dtype=bool), 1)
color_map = sns.color_palette("vlag", as_cmap=True)
var_data['short diagnosis'] = var_data['diagnosis'].apply(lambda x: x.replace('+Other', ''))

groups = list(var_data['short diagnosis'].unique())

fig, axs = plt.subplots(4, 4, figsize=(25, 20), tight_layout=True)
fig.suptitle('Variance correlation difference (row - column)', fontsize = 25, y = 1)

for g1 in range(len(groups)):
        for g2 in range(len(groups)):

                group_one_data = var_data[var_data['short diagnosis'] == groups[g1]][roi_names].corr()
                
                group_two_data = var_data[var_data['short diagnosis'] == groups[g2]][roi_names].corr()
                
                values = group_one_data.to_numpy()-group_two_data.to_numpy()
                max_value = values.flatten().max()
                
                #Create subplot
                sns.heatmap(group_one_data-group_two_data,
                        vmin = 1,#max_value,
                        vmax = -1, #max_value,
                        mask = coor_mask,
                        cmap = color_map, 
                        ax = axs[g1][g2])
                axs[g1][g2].set_title(f"{groups[g1]} - {groups[g2]}")